In [87]:
import pandas as pd
from selenium import webdriver
import time as t
from bs4 import BeautifulSoup


In [88]:

def import_ibv(url="https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br"):
    # definindo a url da b3
    
    # iniciando o webdriver, que deve estar com o driver na pasta do arquivo para funcionar, nesse caso será utilizado do Chrome
    browser = webdriver.Chrome()

    # Entrando na url informada
    browser.get(url)

    # Encontrando o elemento no html para filtrarmos os dados que precisamos
    browser.find_element_by_id('segment').send_keys('Setor de Atuação')

    # Aguardando 2 segundos para a página atualizar
    t.sleep(2)

    # Agora selecionando o o menu para podermos ver todas as ações disponíveis
    browser.find_element_by_id('selectPage').send_keys('120')

    # Aguardando 1 segundo para o página atualizar
    t.sleep(1)

    # IMportando o html inteiro para o Python na variável html
    html = browser.page_source

    # Fechando o navegador
    browser.close()

    # Usando o beatiful soup e convertendo o arquivo para podermos selecionar os dados que precisamos
    bs = BeautifulSoup(html, 'html.parser')

    # Criando as listas onde iremos armazenar os dados do html
    setor, cod, acao, tipo, qtde_t, part, part_a = [], [], [], [], [], [], []

    # Usando a função find_all para selecionarmos apenas 'td' que contém os dados que usaremos e armazenando eles na variável linhas
    linhas = bs.find_all('td')

    # Criando uma variável que será usada no while loop abaixo
    i = 0

    # While loop para podermos armazenar os dados em suas respectivas colunas, onde:
    # temos 7 colunas então pegamos o primeiro elemento 0 e somando 1 até chegarmos ao último elemento para a coluna [i+6] = ao setimo elemento
    # por fim somando 7 para pularmos para a próxima linhas e seguir a lógica
    # o loop só irá para quando o número de linhas for maior do que o que os dados apresentados, para isso usados o lógica len(linhas) que nos entrega a quantia de dados
    # e como i é somado de 7 a cada iteração o loop só irá para quando todos os dados forem indexidaodos nas suas devidas colunas
    while i < (len(linhas)):
        setor.append(linhas[i].text)
        cod.append(linhas[i+1].text)
        acao.append(linhas[i+2].text)
        tipo.append(linhas[i+3].text)
        qtde_t.append(linhas[i+4].text)
        part.append(linhas[i+5].text)
        part_a.append(linhas[i+6].text)
        i += 7

    # Criando um df para armazearmos os dados
    df = pd.DataFrame({'Setor': setor[:-1],
                       'Acao': cod[:-1],
                       'Empresa': acao[:-1],
                       'Qntd_teorica': qtde_t[:-1],
                       'Part_%': part[:-1]})  # filtramos para eliminar as últimas linhas, pois a b3 entrega uma soma ao final que não queremos
    # Separando os setores e subsetores em colunas
    df['SubSetor'] = df['Setor'].apply(lambda s: s[s.rfind('/')+1:].strip())
    df['Setor'] = df['Setor'].apply(lambda s: s[:s.rfind('/')])

    # Convertendo os valores de string para int
    df['Qntd_teorica'] = df['Qntd_teorica'].apply(lambda s: s.replace(".", ""))
    df['Qntd_teorica'] = pd.to_numeric(df['Qntd_teorica'])
    df['Part_%'] = df['Part_%'].apply(lambda s: s.replace(",", ""))
    df['Part_%'] = pd.to_numeric(df['Part_%'])/1000
    df.sort_values('Part_%', ascending=False, inplace=True)

    # Criando uma colunas com a % acumulada
    df['Part_%_acum'] = df['Part_%'].cumsum()

    # Resetando o index
    df.reset_index(drop=True, inplace=True)

    # fim
    return df


In [ ]:

df = import_ibv()

print(df)


In [ ]:
df = df.to_csv('dados_dab3.csv')

In [ ]:
df = pd.read_csv("dados_dab3_Car_do_dia.csv", index_col=0)
df.info()

In [ ]:
def import_ibv2(url="https://sistemaswebb3-listados.b3.com.br/indexPage/theorical/IBOV?language=pt-br"):
    
    browser = webdriver.Chrome()
    browser.get(url)
    t.sleep(2)
    browser.find_element_by_id('selectPage').send_keys('120')
    t.sleep(2)
    html = browser.page_source
    browser.close()
    bs = BeautifulSoup(html, 'html.parser')
    linhas = bs.find_all('td')
    linhas = linhas
    #print(len(linhas))
    #print(linhas)
    i = 0
    lista_act = []
    while i < (len(linhas)):
        lista_act.append(linhas[i].text)
        i += 1
    codigo = []
    j = 0
    while j < (len(lista_act)-8):
        codigo.append(lista_act[j])
        j+=5
    j=1
    acao = []
    while j < (len(lista_act)-5):
        acao.append(lista_act[j])
        j+=5
    j=2
    tipo = []
    while j < (len(lista_act)-6):
        tipo.append(lista_act[j])
        j+=5
    qtd_t = []
    j=3
    while j < (len(lista_act)-7):
        qtd_t.append(lista_act[j])
        j +=5
    j=4
    part_p = []
    while j < (len(lista_act)-6):
        part_p.append(lista_act[j])
        j+=5 
    df = pd.DataFrame({
        'Codigo': codigo,
        'Acao': acao,
        'Tipo': tipo,
        'Qtde_Teo': qtd_t,
        'Part_%': part_p,
    })
    df['Qtde_Teo'] = df['Qtde_Teo'].map(lambda t: t.replace(".", ""))
    df['Qtde_Teo'] = df['Qtde_Teo'].map(lambda qtd: float(qtd))
    df['Part_%'] = df['Part_%'].map(lambda t: t.replace(",", "."))
    df['Part_%'] = df['Part_%'].map(lambda t: float(t))

    
    return df


In [ ]:
df2 = import_ibv2()

In [ ]:
df2 = pd.read_csv("dados_dab3_jan_a_Abr.csv", index_col=0)

In [ ]:
df2.info()

In [ ]:
df2[df2['Codigo'] == 'ABEV3']

In [ ]:
df2['Qtde_Teo'] = df2['Qtde_Teo'].map(lambda t: t.replace(".", ""))
df2['Qtde_Teo'] = df2['Qtde_Teo'].map(lambda qtd: float(qtd))

In [ ]:
df2['Part_%'] = df2['Part_%'].map(lambda t: t.replace(",", "."))
df2['Part_%'] = df2['Part_%'].map(lambda t: float(t))

In [ ]:
df2['Codigo']

In [ ]:
df2 = df2.to_csv("dados_dab3_jan_a_Abr.csv")

In [89]:
def import_ibv3(url="https://sistemaswebb3-listados.b3.com.br/indexPage/preview/IBOV?language=pt-br"):

    browser = webdriver.Chrome()
    browser.get(url)
    t.sleep(2)
    browser.find_element_by_id('selectPage').send_keys('120')
    t.sleep(2)
    html = browser.page_source
    browser.close()
    bs = BeautifulSoup(html, 'html.parser')
    linhas = bs.find_all('td')
    linhas = linhas
    #print(len(linhas))
    #print(linhas)
    i = 0
    lista_act = []
    while i < (len(linhas)):
        lista_act.append(linhas[i].text)
        i += 1
    codigo = []
    j = 0
    while j < (len(lista_act)-8):
        codigo.append(lista_act[j])
        j += 5
    j = 1
    acao = []
    while j < (len(lista_act)-5):
        acao.append(lista_act[j])
        j += 5
    j = 2
    tipo = []
    while j < (len(lista_act)-6):
        tipo.append(lista_act[j])
        j += 5
    qtd_t = []
    j = 3
    while j < (len(lista_act)-7):
        qtd_t.append(lista_act[j])
        j += 5
    j = 4
    part_p = []
    while j < (len(lista_act)-6):
        part_p.append(lista_act[j])
        j += 5
    df = pd.DataFrame({
        'Codigo': codigo,
        'Acao': acao,
        'Tipo': tipo,
        'Qtde_Teo': qtd_t,
        'Part_%': part_p,
    })
    
    df['Qtde_Teo'] = df['Qtde_Teo'].map(lambda t: t.replace(".", ""))
    df['Qtde_Teo'] = df['Qtde_Teo'].map(lambda qtd: float(qtd))
    df['Part_%'] = df['Part_%'].map(lambda t: t.replace(",", "."))
    df['Part_%'] = df['Part_%'].map(lambda t: float(t))
    
    return df

In [ ]:
df3 = import_ibv3()

In [ ]:
df3

In [84]:
df3 = df3.to_csv('dados_dab3_mai_a_Ago.csv')


In [85]:
df3 = pd.read_csv('dados_dab3_mai_a_Ago.csv', index_col=0)
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91 entries, 0 to 90
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Setor         91 non-null     object 
 1   Acao          91 non-null     object 
 2   Empresa       91 non-null     object 
 3   Qntd_teorica  91 non-null     int64  
 4   Part_%        91 non-null     float64
 5   SubSetor      91 non-null     object 
 6   Part_%_acum   91 non-null     float64
dtypes: float64(2), int64(1), object(4)
memory usage: 5.7+ KB


In [86]:
df3['Qtde_Teo'] = df3['Qtde_Teo'].map(lambda t: t.replace(".", ""))
df3['Qtde_Teo'] = df3['Qtde_Teo'].map(lambda qtd: float(qtd))


KeyError: 'Qtde_Teo'

In [ ]:
df3['Part_%'] = df3['Part_%'].map(lambda t: t.replace(",", "."))
df3['Part_%'] = df3['Part_%'].map(lambda t: float(t))

In [ ]:
df3

In [ ]:
df3 = df3.to_csv("dados_dab3_mai_a_Ago.csv")